# Using PyTorch Inside a Django App
> "Building a simple image classification web app using Django and a pretrained PyTorch model."

- hide: true
- toc: true
- branch: master
- badges: true
- comments: true
- categories: [python, machine learning, pytorch, vision, classification, django, heroku]
- image: images/logos/pytorch.png

In this blog post, I build a simple image classification app using a pre-trained model in PyTorch.
I deploy this image classification model inside a Django web app on Heroku.


This is very much related to the [PyTorch guide on deployment with Flask](https://pytorch.org/tutorials/intermediate/flask_rest_api_tutorial.html).
Here, I show an alternative using Django, which is not as light-weight but contains more features built-in than Flask.
For more information on differences between Django and Flask, see [this website](https://hackr.io/blog/flask-vs-django).

Also, refer to my related blog posts on [building and deploying Django apps on Heroku](https://stefanbschneider.github.io/blog/django-heroku)
as well as [getting started with PyTorch](https://stefanbschneider.github.io/blog/pytorch-getting-started).


## Basic Django Setup

Install:

```
pip install django
```

Create the project and a Django app `image_classification`:

```
django-admin startproject pytorch_django
cd pytorch_django
python manage.py startapp image_classification
```

Inside `settings.py`, add `'image_classification.apps.ImageClassificationConfig'` to the `INSTALLED_APPS` list.

To verify, that there are no errors yet, start the Django dev server:

```
python manage.py runserver
```

Go to `localhost:8000`:

![](pytorch-django/django-setup-complete.png)


## URL setup

Then prepare a separate `image_classification/urls.py` for the URLs of the image classification app:
```python
from django.urls import path, include
from django.conf import settings
from django.conf.urls.static import static
from . import views

app_name = 'image_classification'
urlpatterns = [
    # two paths: with or without given image
    path('', views.index, name='index'),
] + static(settings.MEDIA_URL, document_root=settings.MEDIA_ROOT)
```
And link these URLs to the project's URLs in `pytorch_django/urls.py`:
```python
urlpatterns = [
    path('', include('image_classification.urls')),
    path('admin/', admin.site.urls),
]
```
For the simple app here, I only need the default URL, which points to my `index` view,
which I implement next.


## Image Upload and Display

After this initial setup, I need to build a simple view and template that allows users to upload an image for classification.
As usual, user input is handled through a Django form, here just consisting of a single `ImageField` (`image_classification/forms.py`):
```python
from django import forms

class ImageUploadForm(forms.Form):
    image = forms.ImageField()
```
Here, I just want to display the uploaded image and pass it to the PyTorch model for classification.
I do not want to (even temporarily) store it in the file system/disk.
Hence, inside the view (`image_classification/views.py`), I get the image from the form, get its byte representation (for PyTorch later)
and create an image URI for displaying the image later (see [StackOverflow](https://stackoverflow.com/a/40568024/2745116)):

```python
def index(request):
    image_uri = None
    predicted_label = None

    if request.method == 'POST':
        form = ImageUploadForm(request.POST, request.FILES)
        if form.is_valid():
            # passing the image as base64 string to avoid storing it to DB or filesystem
            image = form.cleaned_data['image']
            image_bytes = image.file.read()
            encoded_img = base64.b64encode(image_bytes).decode('ascii')
            image_uri = 'data:%s;base64,%s' % ('image/jpeg', encoded_img)

            # get predicted label
            predicted_label = get_prediction(image_bytes)

    else:
        form = ImageUploadForm()

    # pass the form, image URI, and predicted label to the template to be rendered
    context = {
        'form': form,
        'image_uri': image_uri,
        'predicted_label': predicted_label,
    }
    return render(request, 'image_classification/index.html', context)
```
TODO: Note that the `get_prediction()` function is implemented below.


To display the uploaded image and prediction by the PyTorch model, create a template `image_classification/templates/image_classification/index.html`:
```html
<h1>PyTorch Image Classification App</h1>
<p>A pretrained PyTorch DenseNet model will try to classify the selected image according to ImageNet labels.</p>

<form method="post" enctype="multipart/form-data" style="margin-top: 50px; margin-bottom: 50px;">
    {% csrf_token %}
    {{ form }}
    <button type="submit" class="btn btn-primary">Upload</button>
</form>

{% if image_uri is not None %}
    <figure class="figure">
        <img src="{{ image_uri }}" class="img-fluid" alt="Uploaded image" style="max-width: 500px; max-height: 500px;">
        <figcaption class="figure-caption">Figure selected for classification.</figcaption>
    </figure>

    {% if predicted_label is not None %}
        <div class="alert alert-primary" role="alert">
            Predicted label: {{ predicted_label }}
        </div>
    {% endif %}
{% endif %}
```
The uploaded image uses the saved and passed image URI from before and does not need to load any image from disk.
This template relies on some Bootstrap styling (see [my corresponding Blog post](https://stefanbschneider.github.io/blog/django-bootstrap)),
but it's of course possible to omit that.


TODO: Restructure --> first setup of both djang oand pytorch, 2. dummy django app, 3. pytorch logic, 4. django integration and view, 5. heroku

FIXME: heroku build breaks...

## PyTorch

### Installation

PyTorch classification:

```
pip install torchvision
```

### Image Classification


## Deployment on Heroku

Heroku Deployment:

* Register/Login --> Create new app --> Connect to GitHub (Need GitHub repo!)
* Enable automatic deploys
* Secrets:
DJANGO_SETTINGS_MODULE: project.prod_settings
DJANGO_SECRET_KEY: <randomly-generated-secret-key>

Also prepare `prod_settings`

* add `runtime.txt` and `Procfile`
* Install `whitenoise` and add `'whitenoise.middleware.WhiteNoiseMiddleware',` to middleware in `settings.py` for static file serving

Add to `settings.py`:
```
STATIC_URL = '/static/'
# path to where static files are copied for deployment (eg, for heroku)
STATIC_ROOT = os.path.join(BASE_DIR, 'staticfiles')
# location of static files in local development: https://learndjango.com/tutorials/django-favicon-tutorial
STATICFILES_DIRS = [os.path.join(BASE_DIR, 'static'),]
```


